In [57]:
# Loading libaries and Dataset 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


df=pd.read_csv('../input/nikola-stocks-a-great-big-phony/NKLA.csv')
df

# Loading Dataset 

In [58]:
df.info()

In [59]:
df.isnull().sum()

# Observation - No null Values

In [60]:
# Make sure that you have all these libaries available to run the code successfully
from pandas_datareader import data
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import datetime as dt
import urllib.request, json
import os
import numpy as np
import tensorflow as tf # This code has been tested with TensorFlow 1.6
import math
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM


# Installiing Important Library

In [61]:
pip install plotly


In [62]:
import plotly.graph_objects as go
sns.set_context('poster', font_scale=0.8)
fig = go.Figure(data=go.Scatter(x=df['Date'],y=df['Close'], mode='lines'))
fig.show()


# Trading Volume To The Stock Chart


In [63]:
from plotly.subplots import make_subplots

fig2 = make_subplots(specs=[[{"secondary_y": True}]])
fig2.add_trace(go.Scatter(x=df['Date'],y=df['Close'],name='Price'),secondary_y=False)
fig2.add_trace(go.Bar(x=df['Date'],y=df['Volume'],name='Volume'),secondary_y=True)
fig2.show()


In [64]:
#fig = go.Figure(data=go.Scatter(x=df.index,y=df['Close'], mode='lines+markers'))
#fig.show()


In [65]:
df['diff'] = df['Close'] - df['Open']
df.loc[df['diff']>=0, 'color'] = 'green'
df.loc[df['diff']<0, 'color'] = 'red'


We’ll draw a simple indicator 20 Day Moving Average here to show the concept, theoretically, we can plot any indicator on the chart.



# 20 Days  Indicators

In [66]:
fig3 = make_subplots(specs=[[{"secondary_y": True}]])
fig3.add_trace(go.Candlestick(x=df.index,
                              open=df['Open'],
                              high=df['High'],
                              low=df['Low'],
                              close=df['Close'],
                              name='Price'))
fig3.add_trace(go.Scatter(x=df.index,y=df['Close'].rolling(window=20).mean(),marker_color='blue',name='20 Day MA'))
fig3.add_trace(go.Bar(x=df.index, y=df['Volume'], name='Volume', marker={'color':df['color']}),secondary_y=True)
fig3.update_yaxes(range=[0,700000000],secondary_y=True)
fig3.update_yaxes(visible=False, secondary_y=True)
fig3.update_layout(xaxis_rangeslider_visible=False)  #hide range slider
fig3.update_layout(title={'text':'NKLA', 'x':0.5})
fig3.show()


In [67]:
# Ploting Close Price History using matplotlib¶
plt.figure(figsize=(16,8))
plt.title('Close Price History')
plt.plot(df['Close'])
#ax=sns.lineplot(data=df, x='timestamp',y='close', color="blue");
plt.xlabel('Date',fontsize=18)
plt.ylabel('Close Price INR',fontsize=18)
plt.show()


# Use the Open Stock Price Column to Train Your Model.


# Normalizing the Dataset.


In [68]:
# Converting data to a numpy array¶
data = df.filter(['Close'])
dataset = data.values
training_data_len = math.ceil( len(dataset) *.8) 
dataset

In [69]:
# Transforming the dataset array to range between 0 and 1¶
scaler = MinMaxScaler(feature_range=(0, 1)) 
scaled_data = scaler.fit_transform(dataset)


# Creating X_train and y_train Data Structures.


In [70]:
train_data = scaled_data[0:training_data_len  , : ]
x_train=[]
y_train = []
for i in range(60,len(train_data)):
    x_train.append(train_data[i-60:i,0])
    y_train.append(train_data[i,0])


# Reshape the Data.


In [71]:
# Spliting data for training and testing¶
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))


# Building the Model by Importing the Crucial Libraries and Adding Different Layers to LSTM.


In [72]:
# Building a LSTM Model for Stock Market Prediction¶
model = Sequential()
model.add(LSTM(units=50, return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))


# Fitting the Model.


In [73]:
model.compile(optimizer='adam', loss='mean_squared_error')


In [74]:
test_data = scaled_data[training_data_len - 60: , : ]#Create the x_test and y_test data sets
x_test = []
y_test =  dataset[training_data_len : , : ] #Get all of the rows from index 1603 to the rest and all of the columns (in this case it's only column 'Close'), so 2003 - 1603 = 400 rows of data
for i in range(60,len(test_data)):
    x_test.append(test_data[i-60:i,0])


In [75]:
x_test = np.array(x_test)


In [76]:
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))


In [77]:
predictions = model.predict(x_test) 
predictions = scaler.inverse_transform(predictions)


# Extracting the Actual Stock Prices of Jun-2022.


In [78]:
# Finding the root mean squared error¶
rmse=np.sqrt(np.mean(((predictions- y_test)**2)))
rmse


# Plotting the Actual and Predicted Prices for NKLA.


In [79]:
# Plotting the predicted values¶
train = data[:training_data_len]
display = data[training_data_len:]
display['Predictions'] = predictions#Visualize the data
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price INR', fontsize=18)
plt.plot(train['Close'])
plt.plot(display['Close'])
plt.plot(display['Predictions'])
plt.legend(['Train', 'Val', 'Predictions'], loc='upper right')
plt.show()
